In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from my_functions import *
import csv
from colorplot import color_cut, make_colorplot, plot_selection
from LumFunc_miniJPAS import LumFunc_hist

In [ ]:
import warnings
warnings.filterwarnings("ignore", message="divide by zero encountered in true_divide")
warnings.filterwarnings("ignore", message="Mean of empty slice")

In [ ]:
## Function that loads from a csv file the DualABMag minijpas catalog with associated photoz,
## odds, and GAIA apparent move data.
def load_cat_photoz_gaia(filename):
    with open(filename, mode='r') as csvfile:
        rdlns = csv.reader(csvfile, delimiter=',')
        next(rdlns, None)
        next(rdlns, None)
        
        number = []
        mag = []
        mag_err = []
        flags = []
        mflags = []
        photoz = []
        odds = []
        parallax = []
        parallax_err = []
        pmra = []
        pmra_err = []
        pmdec = []
        pmdec_err = []
        
        for line in rdlns:
            number.append(line[0])
            mag.append(line[1].split())
            mag_err.append(line[2].split())
            flags.append(line[3].split())
            mflags.append(line[4].split())
            photoz.append(line[5])
            odds.append(line[6])
            parallax.append(line[7])
            parallax_err.append(line[8])
            pmra.append(line[9])
            pmra_err.append(line[10])
            pmdec.append(line[11])
            pmdec_err.append(line[12])
            
    columns = [
        number, mag, mag_err,
        flags, mflags, photoz,
        odds, parallax, parallax_err,
        pmra, pmra_err, pmdec,
        pmdec_err
    ]
    cat_keys = [
        'number', 'mag', 'mag_err', 'flags',
        'mflags', 'photoz', 'odds', 'parallax',
        'parallax_err', 'pmra', 'pmra_err',
        'pmdec', 'pmdec_err'
    ]
    cat_types = [
        int, float, float, int,
        int, float, float, float,
        float, float, float, float,
        float
    ]
    cat = {}
    
    for col,key in zip(columns, cat_keys):
        cat[key] = np.array(col)
    # Substitute empty values by NumPy NaN
    for k,t in zip(cat.keys(), cat_types):
        cat[k][np.where(cat[k] == '')] = np.nan
        cat[k] = cat[k].astype(t)
        
    # Remove flagged sources
    flags_arr = np.sum(cat['flags'], axis = 1) + np.sum(cat['mflags'], axis = 1)
    mask_flags = flags_arr == 0
    for k in cat.keys():
        cat[k] = cat[k][mask_flags]
    
    return cat

In [ ]:
## Function to compute the probability of a reliable excess for nb_ind and bb_ind
def p_lya_bbnb(cat, bb_ind, nb_ind):
    bb_m_zero = cat['mag'][:, bb_ind]
    nb_m_zero = cat['mag'][:, nb_ind]
    bb_e_zero = cat['mag_err'][:, bb_ind]
    nb_e_zero = cat['mag_err'][:, nb_ind]
    odds = cat['odds']
    
    # Mask sources with negative flux values (i.e. m = 99)
    mask_neg_flux = (bb_m_zero < 90.) & (nb_m_zero < 90.)
    # Mask sources with too high photoz odds
    mask_pz_odds = (odds < 0.98)
    # Mask sources with proper motion according to Gaia (sigma > 3)
    parallax_sn = np.abs(cat['parallax'] / cat['parallax_err'])
    pmra_sn = np.abs(cat['pmra'] / cat['pmra_err'])
    pmdec_sn = np.abs(cat['pmdec'] / cat['pmdec_err'])
    mask_pmotion = (
        (np.sqrt(parallax_sn**2 + pmra_sn**2 + pmdec_sn**2) < 27.**0.5)
        | ( np.isnan(parallax_sn) | np.isnan(pmra_sn) | np.isnan(pmdec_sn) )
    )
    # Mask sources with SNR < 5
    mask_snr = (bb_m_zero/bb_e_zero >= 5) & (nb_m_zero/nb_e_zero >= 5)
#     # Mask flagged
#     mask_flag = ((np.sum(cat['flags'], axis = 1) + np.sum(cat['mflags'], axis = 1)) == 0)
    
    mask_total = ( mask_neg_flux & mask_pz_odds & mask_pmotion & mask_snr)
    print('\nmask_total:    {}'.format(len(np.where(~mask_total)[0])))
    print('N sources:     {}'.format(len(np.where(mask_total)[0])))
    
    p_bbnb = np.zeros(bb_m_zero.shape)
    mu = bb_m_zero[mask_total] - nb_m_zero[mask_total]
    sigma = (bb_e_zero[mask_total]**2 + nb_e_zero[mask_total]**2)**0.5
    m_ew = color_cut(30, nb_ind) + np.nanmedian(mu)
    p_bbnb[mask_total] = 0.5 - 0.5*erf((m_ew - mu) / (np.sqrt(2)*sigma))
    
    return p_bbnb

In [ ]:
## Load the catalog
cat = load_cat_photoz_gaia('csv/minijpas.magAB_photoz_gaia.csv')
# cat_f = load_cat_photoz_gaia('csv/minijpas.Flambda_photoz_gaia.csv')

In [ ]:
# Define the array of the filters we are going to use // gSDSS band
nb_ind_arr = [8, 9, 10, 11, 12, 13, 14, 15]
bb_ind = -3

In [ ]:
p_line = []
for nb_ind in nb_ind_arr:
    p_line.append(p_lya_bbnb(cat, bb_ind, nb_ind))

In [ ]:
hist_arr = []
hist_arr_w = []
volume_arr = []
w_central_arr = central_wavelength(load_tcurves(load_filter_tags()))

for nb_ind, p in zip(nb_ind_arr, p_line):
    nb_m = cat['mag'][:, nb_ind]
    bb_m = cat['mag'][:, bb_ind]
    nb_e = cat['mag_err'][:, nb_ind]
    bb_e = cat['mag_err'][:, bb_ind]

    colorcut = color_cut(20, nb_ind)

    selection, = np.where(p > 0.997)
    selection_w, = np.where(p > 0.95)
    
#     #########################################
        
#     selection_f = []
#     for i in selection_w:
#         selection_f, = np.where(cat['number'][i] == cat_f['number'])
#         plot_selection(
#             selection_f, nb_ind, 'selected_sources/a',
#             cat_f['mag']*1e-19, cat_f['mag_err']*cat_f['mag']*1e-19
#         )
    
#     #########################################

    make_colorplot(nb_m, bb_m, nb_e, bb_e, [], colorcut, p)
    make_colorplot(nb_m, bb_m, nb_e, bb_e, selection, colorcut, [])
    
    w_pivot = w_central_arr[nb_ind]
    w_pivot_bb = w_central_arr[bb_ind]
    w_fwhm = nb_fwhm(load_tcurves(load_filter_tags()), nb_ind, True)
    f_line = mag_to_flux(nb_m[selection], w_pivot) - mag_to_flux(bb_m[selection], w_pivot_bb)
    f_line_w = mag_to_flux(nb_m[selection_w], w_pivot) - mag_to_flux(bb_m[selection_w], w_pivot_bb)
    n_bins = 10
    
    x, hist, volume, bin_width = LumFunc_hist(f_line, w_pivot, w_fwhm, n_bins, 43.5, 44.5)
    x, hist_w, volume, bin_width = LumFunc_hist(f_line_w, w_pivot, w_fwhm, n_bins, 43.5, 44.5, p)
    hist_arr.append(hist)
    hist_arr_w.append(hist_w)
    volume_arr.append(volume)

In [ ]:
## Load HETDEX LumFunc data (Zhang 2021)
filepath = 'csv/HETDEX_LumFunc.csv'
HETDEX_LF = []
with open(filepath, mode='r') as csvfile:
    rdlns = csv.reader(csvfile, delimiter=',')
    for line in rdlns:
        HETDEX_LF.append(line)
HETDEX_LF = np.array(HETDEX_LF).astype(float)

In [ ]:
## Make the LF
y = np.sum(hist_arr, axis = 0)/np.sum(volume_arr)/bin_width
errors = np.sqrt(np.sum(hist_arr, axis = 0))/np.sum(volume_arr)/bin_width
print(np.sum(hist_arr, axis = 0))
fig, ax = plt.subplots(figsize=(10,8))

ax.plot(HETDEX_LF[:,0], HETDEX_LF[:,1], 's', markersize = 10, label = 'Zhang (2021)')
ax.errorbar(HETDEX_LF[:,0], HETDEX_LF[:,1], yerr = HETDEX_LF[:,2] - HETDEX_LF[:,1], fmt = 'none')

ax.plot(x, y, '.', markersize=20, c='orange', label = 'My points')
ax.errorbar(x, y, yerr = errors, fmt = 'none', c='orange')

ax.set_yscale('log')
ax.set_ylabel(r'$\Phi$ [Mpc$^{-3}\Delta\log L^{-1}$]', size=15)
ax.set_xlabel(r'$\log_{10}(L_{\mathrm{Ly}\alpha}\,/\,\mathrm{erg}\,\mathrm{s}^{-1})$', size=15)
ax.legend()
plt.show()

In [ ]:
## Make the LF
y = np.sum(hist_arr_w, axis = 0)/np.sum(volume_arr)/bin_width
errors = np.sqrt(np.sum(hist_arr, axis = 0))/np.sum(volume_arr)/bin_width
print(np.sum(hist_arr_w, axis = 0))
fig, ax = plt.subplots(figsize=(10,8))

ax.plot(HETDEX_LF[:,0], HETDEX_LF[:,1], 's', markersize = 10, label = 'Zhang (2021)')
ax.errorbar(HETDEX_LF[:,0], HETDEX_LF[:,1], yerr = HETDEX_LF[:,2] - HETDEX_LF[:,1], fmt = 'none')

ax.plot(x, y, '.', markersize=20, c='orange', label = 'My points')
ax.errorbar(x, y, yerr = errors, fmt = 'none', c='orange')

ax.set_yscale('log')
ax.set_ylabel(r'$\Phi$ [Mpc$^{-3}\Delta\log L^{-1}$]', size=15)
ax.set_xlabel(r'$\log_{10}(L_{\mathrm{Ly}\alpha}\,/\,\mathrm{erg}\,\mathrm{s}^{-1})$', size=15)
ax.legend()
plt.show()